# Predicción de Precios de Coches Usados - Modelo de Machine Learning

## Análisis y procesamiento de datos

### Introduccion
La estimación del precio de un coche de segunda mano es un problema complejo, ya que depende de múltiples factores como el año, el kilometraje, la marca y otras características del vehículo. Para abordar esta tarea, se ha recopilado un conjunto de datos con ventas históricas de coches usados, incluyendo el precio real de venta. A partir de esta información, se aplicarán técnicas de Machine Learning para analizar y procesar los datos, entrenar modelos predictivos y evaluar su desempeño, con el objetivo de sugerir precios en portales de compraventa online o realizar tasaciones automáticas.

### Objetivo
Desarrollar e implementar un modelo de aprendizaje supervisado capaz de predecir el precio de venta de coches de segunda mano, evaluando su desempeño mediante el error absoluto medio (MAE) en el conjunto de validación, considerando satisfactorio un MAE inferior a 3,000 €, con el fin de generar predicciones precisas útiles para la tasación automática o la sugerencia de precios en portales de compraventa online.

In [22]:
# Importamos las librerias necesarias
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn.preprocessing as prep
%matplotlib inline

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error , mean_absolute_percentage_error
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.metrics import precision_score , recall_score , confusion_matrix
from sklearn.metrics import roc_curve , auc
from sklearn.tree import DecisionTreeClassifier , DecisionTreeRegressor
from sklearn.tree import plot_tree , export_text
from sklearn.svm import SVR
from sklearn.preprocessing import MinMaxScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import roc_auc_score
from sklearn.neural_network import MLPClassifier
#HIperparametros
from sklearn.model_selection import GridSearchCV
from sklearn.feature_selection import SelectKBest, chi2, f_regression
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import VotingClassifier
from sklearn.svm import SVC

from sklearn.neural_network import MLPRegressor
from sklearn.pipeline import Pipeline
from joblib import dump






### Cargar datos
En este proyecto utilizaremos Pandas y NumPy para la manipulación y análisis de datos, Matplotlib y Seaborn para la visualización gráfica, y Scikit-Learn para el desarrollo y evaluación de los modelos de machine learning.

In [23]:
train_data = pd.read_csv("data/dataset_coches_train.csv")
test_data= pd.read_csv("data/dataset_coches_test.csv")

### Explorar datos iniciales
1. Informacion General de los datos

En primer lugar a ver qué información contiene el conjunto de datos : el tamaño,filas y columnas , si hay registros duplicados o ausentes, etc.

In [24]:

print("============== Primeras 5 filas ================")
print(train_data.head()) 
print("============= Últimas 5 filas ====================")
print(train_data.tail())

============== Primeras 5 filas ================
      ID     marca    modelo  fecha tipo_cambio  total_km tipo_combustible  \
0  13207    hyundi  Santa Fe   2019   Semi-Auto      4223           Diesel   
1  17314  vauxhall       GTC   2015      Manual     47870           Diesel   
2  12342      audi       RS4   2019   Automatic      5151           Petrol   
3  13426        vw  Scirocco   2016   Automatic     20423           Diesel   
4  16004     skoda     Scala   2020   Semi-Auto      3569           Petrol   

   consumo  tipo_motor   tasa  precio  
0     5.91         2.2  145.0   31995  
1     3.91         2.0  125.0    7700  
2     8.08         2.9  145.0   58990  
3     4.08         2.0   30.0   12999  
4     4.99         1.0  145.0   16990  
============= Últimas 5 filas ====================
         ID marca    modelo  fecha tipo_cambio  total_km tipo_combustible  \
4955  16898  merc  GL Class   2015   Automatic     24314           Diesel   
4956  14416   bmw  6 Series   2017   

In [25]:
print("======== Informacion general del Dataframe =========")
train_data.info()
print("=========== Resumen estadistico de las variables ========")
train_data.describe(include="all")


======== Informacion general del Dataframe =========
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4960 entries, 0 to 4959
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   ID                4960 non-null   int64  
 1   marca             4960 non-null   object 
 2   modelo            4960 non-null   object 
 3   fecha             4960 non-null   int64  
 4   tipo_cambio       4812 non-null   object 
 5   total_km          4960 non-null   int64  
 6   tipo_combustible  4960 non-null   object 
 7   consumo           4564 non-null   float64
 8   tipo_motor        4960 non-null   float64
 9   tasa              4960 non-null   float64
 10  precio            4960 non-null   int64  
dtypes: float64(3), int64(4), object(4)
memory usage: 426.4+ KB
=========== Resumen estadistico de las variables ========


,ID,marca,modelo,fecha,tipo_cambio,total_km,tipo_combustible,consumo,tipo_motor,tasa,precio
count,4960.000000,4960,4960,4960.000000,4812,4960.000000,4960,4564.000000,4960.000000,4960.000000,4960.000000
unique,NaN,9,90,NaN,4,NaN,5,NaN,NaN,NaN,NaN
top,NaN,merc,Arteon,NaN,Manual,NaN,Diesel,NaN,NaN,NaN,NaN
freq,NaN,790,167,NaN,1886,NaN,2989,NaN,NaN,NaN,NaN
mean,15832.446169,NaN,NaN,2016.737903,NaN,24956.286895,NaN,5.388508,2.124375,152.332661,23469.943750
std,2206.717006,NaN,NaN,2.884035,NaN,24443.333662,NaN,3.246347,0.789368,82.403844,16406.714563
min,12002.000000,NaN,NaN,1997.000000,NaN,1.000000,NaN,0.500000,0.000000,0.000000,450.000000
25%,13929.250000,NaN,NaN,2016.000000,NaN,5641.250000,NaN,4.330000,1.600000,145.000000,12000.000000
50%,15840.000000,NaN,NaN,2017.000000,NaN,19000.000000,NaN,4.990000,2.000000,145.000000,18999.000000
75%,17765.750000,NaN,NaN,2019.000000,NaN,36702.000000,NaN,6.080000,2.800000,150.000000,30335.750000


In [26]:
print("=========== Resumen estadistico de las variables ========")
train_data.describe(include="all")


=========== Resumen estadistico de las variables ========


,ID,marca,modelo,fecha,tipo_cambio,total_km,tipo_combustible,consumo,tipo_motor,tasa,precio
count,4960.000000,4960,4960,4960.000000,4812,4960.000000,4960,4564.000000,4960.000000,4960.000000,4960.000000
unique,NaN,9,90,NaN,4,NaN,5,NaN,NaN,NaN,NaN
top,NaN,merc,Arteon,NaN,Manual,NaN,Diesel,NaN,NaN,NaN,NaN
freq,NaN,790,167,NaN,1886,NaN,2989,NaN,NaN,NaN,NaN
mean,15832.446169,NaN,NaN,2016.737903,NaN,24956.286895,NaN,5.388508,2.124375,152.332661,23469.943750
std,2206.717006,NaN,NaN,2.884035,NaN,24443.333662,NaN,3.246347,0.789368,82.403844,16406.714563
min,12002.000000,NaN,NaN,1997.000000,NaN,1.000000,NaN,0.500000,0.000000,0.000000,450.000000
25%,13929.250000,NaN,NaN,2016.000000,NaN,5641.250000,NaN,4.330000,1.600000,145.000000,12000.000000
50%,15840.000000,NaN,NaN,2017.000000,NaN,19000.000000,NaN,4.990000,2.000000,145.000000,18999.000000
75%,17765.750000,NaN,NaN,2019.000000,NaN,36702.000000,NaN,6.080000,2.800000,150.000000,30335.750000


In [27]:
train_data.isnull().sum()


ID                    0
marca                 0
modelo                0
fecha                 0
tipo_cambio         148
total_km              0
tipo_combustible      0
consumo             396
tipo_motor            0
tasa                  0
precio                0
dtype: int64

In [28]:
print(f'Número de valores duplicados: {train_data.duplicated().sum()}')

Número de valores duplicados: 0


El dataset contiene 4960 registros , distribuidos con 11 columnas con sus respectivas caracteristicas , siete de tipo numerico y cuatro de tipo categorico.En un análisis inicial se identifican 9 marcas y 90 modelos distintos, siendo Mercedes-Benz la marca más frecuente.
La antigüedad media de los vehículos corresponde al año 2017, lo que indica que, en general, se trata de coches relativamente recientes.
Por una lado la variable objetivo precio, los valores oscilan entre 450 € y 145.000 €, lo que sugiere la presencia de outliers significativos que deberán analizarse o tratarse antes del modelado.Por otro lado A partir del metodo nulos y duplicados  se identificó que la variable tipo_cambio presenta 148 valores faltantes, mientras que la variable consumo contiene 396 valores faltantes, lo que evidencia la necesidad de aplicar técnicas de imputación o eliminación antes del entrenamiento del modelo y por ultimo no se encontro valores nulos.

In [29]:
train_data["precio"].min()

np.int64(450)

##2.Visualizacion de datos
